In [1]:
import numpy as np
import veloxchem as vlx

In [2]:
ethylene_xyz = """
6

C        0.67759997    0.00000000   0.00000000
C       -0.67759997    0.00000000   0.00000000
H        1.21655197    0.92414474   0.00000000
H        1.21655197   -0.92414474   0.00000000
H       -1.21655197   -0.92414474   0.00000000
H       -1.21655197    0.92414474   0.00000000
"""

In [3]:
molecule = vlx.Molecule.read_xyz_string(ethylene_xyz)
basis = vlx.MolecularBasis.read(molecule, "def2-svp", ostream=None)

nocc = molecule.number_of_alpha_electrons()
norb = basis.get_dimension_of_basis()
nvirt = norb - nocc

print("Number of occupied orbitals :", nocc)
print("Number of virtual orbitals  :", nvirt)
print("Number of molecular orbitals:", norb)

Number of occupied orbitals : 8
Number of virtual orbitals  : 40
Number of molecular orbitals: 48


In [4]:
molecule.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [5]:
scf_drv = vlx.ScfRestrictedDriver()
scf_drv.ostream.mute()

scf_results = scf_drv.compute(molecule, basis)

In [6]:
dipole_drv = vlx.ElectricDipoleIntegralsDriver()
dipole_mats = dipole_drv.compute(molecule, basis)

mu_x_ao = -1.0 * dipole_mats.x_to_numpy()

C = scf_results["C_alpha"]

mu_x = np.einsum("ap, ab, bq -> pq", C, mu_x_ao, C)

In [7]:
lrf_drv = vlx.LinearResponseSolver()
lrf_drv.ostream.mute()

lrf_drv.a_operator = "electric dipole"
lrf_drv.b_operator = "electric dipole"

lrf_drv.a_components = ["x"]
lrf_drv.b_components = ["x"]

lrf_drv.frequencies = [0.0656]

lrf_results = lrf_drv.compute(molecule, basis, scf_results)

In [8]:
Z = -0.5 * (
    lrf_results["solutions"][("x", 0.0656)].get_full_vector(0)
    + lrf_results["solutions"][("x", 0.0656)].get_full_vector(1)
) * np.sqrt(2)

Y = -0.5 * (
    lrf_results["solutions"][("x", 0.0656)].get_full_vector(0)
    - lrf_results["solutions"][("x", 0.0656)].get_full_vector(1)
) * np.sqrt(2)

In [9]:
def vector_to_matrix(Z, Y):

    zymat = np.zeros((norb, norb), dtype=float)

    idx = 0
    for i in range(nocc):
        for a in range(nocc, norb):
            zymat[a, i] = Z[idx]
            zymat[i, a] = Y[idx]
            idx += 1

    return zymat

In [10]:
zymat = vector_to_matrix(Z, Y)

In [11]:
def commutator(A, B):

    return np.matmul(A, B) - np.matmul(B, A)

In [12]:
def expectation_value(A):  # closed shell state

    return 2 * np.einsum("ii ->", A[:nocc, :nocc])

In [13]:
alpha_xx = - expectation_value(commutator(zymat, mu_x))

In [14]:
print(f"Polarizability: {alpha_xx:12.8f}")

Polarizability: -34.84380178


In [15]:
print(f"Reference: {-lrf_results['response_functions'][('x', 'x', 0.0656)]:12.8f}")

Reference:  34.84380178
